In [54]:
import json
import requests

"""
call nchc API for LLM available list
"""
def LLMs():
    twcc_key = "sk-DJ0ji_Mskxs8Mu5CjbyGAw"
    url = "https://medusa-poc.genai.nchc.org.tw/v1/models"
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {twcc_key}"
    }
    models = requests.get(url, headers=headers)
    print(f"\n--- Available Model ---")
    print(models.status_code)
    print("models len=", len(models.json()["data"]))
    for model in models.json()["data"]:
        print(model["id"])
    return models


"""
test each available LLM
"""
def _call_nchc(messages, eval_model):
        twcc_key = "sk-DJ0ji_Mskxs8Mu5CjbyGAw"
        url = "https://medusa-poc.genai.nchc.org.tw/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {twcc_key}",
            "Content-Type": "application/json"
        }
        data = {
            "model": eval_model,
            "messages": messages,
            "max_tokens": 100,
            "temperature": 0,
            "top_p": 0.92
        }
        
        try:
            print(f"[NCHC] Calling model: {eval_model}")
            response = requests.post(url, headers=headers, json=data)
            if response.status_code == 200:
                return {
                    "status": 200,
                    "model": eval_model,
                    "content": response.json()['choices'][0]['message']['content']
                }
            else:
                return {
                    "status": response.status_code,
                    "model": eval_model,
                    "error": response.text
                }
        except Exception as e:
            return {
                "status": "exception",
                "model": eval_model,
                "error": str(e)
            }

models = LLMs()

messages = [
    {"role": "system", "content": "LLM asistance"},
    {"role": "user", "content": "what is the weather?"}
]

print(f"\n--- Call Model ---")
results = []
for model in models.json()["data"]:
    eval_model = model["id"]
    result = _call_nchc(messages, eval_model)
    results.append(result)

print(f"\n--- Status Model ---")
for res in results:
    if res["status"] == 200:
        print("✅ Success:", res['model'])
    else:
        print("❌ Error:", res['model'])


available LLM:
200
models len= 20
TAIDE-LX-7B-Chat
Llama-3.1-TAIDE-LX-8B-Chat
Llama-3.1-70B
Llama-3.3-70B-Instruct
Granite-Guardian-3.1-8B
Phi-3.5-TAIDE-Instruct
Llama3-TAIDE-LX-8B-Chat-Alpha1
Microsoft-Phi-4
Llama-3.1-8B-Instruct
Llama3-TAIDE-LX-70B-Chat
Llama-3.1-405B-Instruct-FP8
Llama-3.1-Nemotron-70B-Instruct
Granite-3.1-8B-Instruct
Llama-3.2-90B-Vision-Instruct
TAIDE/Llava-Phi-3.5
Google-Gemma-2-9B
Ministral-8B-Instruct-2410
TAIDE-LX-70B-Chat
Granite-3.1-3B-A800M-Instruct
Mistral-Small-24B-Instruct-2501

--- Call Model ---
[NCHC] Calling model: TAIDE-LX-7B-Chat
[NCHC] Calling model: Llama-3.1-TAIDE-LX-8B-Chat
[NCHC] Calling model: Llama-3.1-70B
[NCHC] Calling model: Llama-3.3-70B-Instruct
[NCHC] Calling model: Granite-Guardian-3.1-8B
[NCHC] Calling model: Phi-3.5-TAIDE-Instruct
[NCHC] Calling model: Llama3-TAIDE-LX-8B-Chat-Alpha1
[NCHC] Calling model: Microsoft-Phi-4
[NCHC] Calling model: Llama-3.1-8B-Instruct
[NCHC] Calling model: Llama3-TAIDE-LX-70B-Chat
[NCHC] Calling model: L

In [57]:
from services.GenReport import GenReport
from services.EvalCitation import EvalCitation
from services.EvalMetrics import EvalMetrics


async def gen_eval_report(dialogue: str, gen_model: str, eval_model: str, object_type: str):
    # Step 0: add index in dialogue
    reporter = GenReport()
    indexed_dialogue = reporter.add_index_to_indexed_dialogue(dialogue)

    # Step 1: gen report
    report_content = await reporter.summary_report(indexed_dialogue, gen_model, object_type)

    # Step 2: eval report
    citation_evaluator = EvalCitation()
    citation_result = await citation_evaluator.evaluate(report_content, indexed_dialogue, eval_model)

    # Step 3: compute citation
    metrics_computer = EvalMetrics()
    metrics = await metrics_computer.compute(citation_result)
    print("[metrics recall]", metrics['citation_recall'])
    print("[metrics precision]", metrics['citation_precision'])

    return indexed_dialogue, report_content, citation_result, metrics


dialogue = """
使用者問題/回覆1: 我的右眼最近很模糊，之前有做過白內障手術。
系統回覆1: 您好，您的症狀可能需要進一步檢查，建議到醫療中心的門診部。
"""
gen_model = "Llama-3.1-Nemotron-70B-Instruct"
object_type = "Doctor"
eval_model = "Llama-3.3-70B-Instruct"

indexed_dialogue, report_content, citation_result, metrics = await gen_eval_report(
    dialogue, gen_model, eval_model, object_type)

[NCHC] Calling model: Llama-3.1-Nemotron-70B-Instruct
[DEBUG] EvalCitation.py & Eval model received: Llama-3.3-70B-Instruct
[NCHC] Calling model: Llama-3.3-70B-Instruct
[NCHC] Calling model: Llama-3.3-70B-Instruct
[NCHC] Calling model: Llama-3.3-70B-Instruct
[DEBUG] EvalMetrics.py Entry
[metrics recall] 1.0
[metrics precision] 1.0


AttributeError: 'str' object has no attribute 'json'